In [1]:
import jieba
import jieba.analyse
import numpy as np
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号
import seaborn as sns
sns.set()
# from wordcloud import WordCloud
from pyecharts.charts import WordCloud
from imageio import imread
from sklearn.decomposition import PCA
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score
from gensim import corpora, models
import pyecharts.options as opts
from collections import Counter
import math
import jieba.analyse

In [71]:
data = pd.read_csv(r'C:\Users\hai\Desktop\test_py\process\NLP\buffertest.csv', encoding='utf-8')
print(data.shape)
left_aligned_df = data.style.set_properties(**{'text-align': 'left'})
left_aligned_df

(68, 1)


# 数据清洗

## 文本规范化
在计算词频前先将文本进行规范化

- 通过正则表达获取中文。
- 去掉"的", "不"这一类停用词。

In [72]:
# 加载停用词
with open(r'C:\Users\hai\Desktop\test_py\process\NLP\stop_words.txt') as f:
    stopword_list = f.readlines()
    stopword_list = [word.strip() for word in stopword_list]   # 去掉停用词后面的空格符

# 分词，并去掉词前后的空格
def tokenize_text(text):
    tokens = jieba.cut(text)
    tokens = [token.strip() for token in tokens]
    return tokens

# 去掉停用词
def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ''.join(filtered_tokens)
    return filtered_text

# 数据规范化
def normalize_corpus(corpus):
    normalized_corpus = []
    text = remove_stopwords(corpus)
    normalized_corpus.append(text)
    return normalized_corpus

### 添加研究相关词语

In [86]:
add_list = ['维护','风景名胜区','风景','名胜区','景区','划定','地带','规划','内外']
stopword_list += add_list

In [87]:
text = data['content']
# 清洗
text_join = ''.join(text)     # 将每条句子合起来
# 获取中文词语
pattern = '[\u4e00-\u9fa5]'
zhong_ls = re.findall(pattern, text_join)
# print(zhong_ls)
# 合并
zhong = ''.join(zhong_ls)
text_join = normalize_corpus(zhong)
text_ls = jieba.lcut(text_join[0],cut_all=True)
text_ls[:50]    # 显示前10条

['发展',
 '应',
 '风景',
 '风景区',
 '景区',
 '发展',
 '协调',
 '并进',
 '进城',
 '城市',
 '发展',
 '边界',
 '禁止',
 '风景',
 '风景区',
 '景区',
 '区外',
 '外围',
 '保护',
 '禁止',
 '城市',
 '建设',
 '用地',
 '侵占',
 '风景',
 '风景区',
 '景区',
 '用地',
 '协调',
 '边界',
 '鞍山',
 '鞍山市',
 '城镇',
 '空间',
 '农业',
 '空间',
 '生态',
 '空间',
 '中城',
 '城镇',
 '开发',
 '边界',
 '永久',
 '基本',
 '基本农田',
 '农田',
 '保护',
 '红线',
 '生态',
 '保护']

In [89]:
text_dic = {}
for word in text_ls:
    if len(word) == 1: #单个的字符不作为词放入字典(其中包括标点)
        continue
    elif word.isdigit() == True: # 剔除数字
        continue
    elif word in text_dic:
        text_dic[word] +=1 #对于重复出现的词，每出现一次，次数增加1
    else:
        text_dic[word] =1
text_dic_seq = sorted(text_dic.items(),key=lambda x:x[1], reverse=True) #按字典的值降序排序
# text_dic_seq #此行代码便于从全部中选取不需要的关键词
text_dic_seq[:500] # 查看前15个

[('保护', 744),
 ('建设', 422),
 ('外围', 387),
 ('景观', 376),
 ('控制', 329),
 ('风景', 295),
 ('景区', 290),
 ('协调', 290),
 ('风景区', 284),
 ('城市', 274),
 ('环境', 247),
 ('发展', 217),
 ('建筑', 210),
 ('生态', 186),
 ('旅游', 159),
 ('管理', 149),
 ('区域', 146),
 ('用地', 139),
 ('设施', 129),
 ('自然', 120),
 ('风貌', 100),
 ('居民', 99),
 ('污染', 98),
 ('空间', 96),
 ('规划', 96),
 ('服务', 90),
 ('资源', 89),
 ('文化', 88),
 ('影响', 87),
 ('城镇', 83),
 ('绿化', 83),
 ('项目', 78),
 ('周边', 73),
 ('特色', 72),
 ('山体', 72),
 ('破坏', 70),
 ('区外', 68),
 ('功能', 65),
 ('措施', 65),
 ('高度', 62),
 ('关系', 61),
 ('活动', 61),
 ('区内', 60),
 ('保护区', 60),
 ('相关', 59),
 ('利用', 59),
 ('产业', 58),
 ('开发', 57),
 ('布局', 56),
 ('严格', 56),
 ('居民点', 56),
 ('严格控制', 55),
 ('整体', 53),
 ('规模', 52),
 ('道路', 52),
 ('调控', 52),
 ('禁止', 49),
 ('度假', 49),
 ('地区', 49),
 ('引导', 48),
 ('恢复', 48),
 ('观风', 46),
 ('村庄', 46),
 ('总体', 44),
 ('西湖', 42),
 ('服务设施', 42),
 ('土地', 41),
 ('生态环境', 41),
 ('绿地', 41),
 ('休闲', 41),
 ('控制区', 41),
 ('面积', 40),
 ('节点', 39),
 ('重点', 39),
 ('片区',

In [90]:
result = pd.DataFrame(text_dic_seq)
# result.to_csv('C://Users//hai//Desktop//text_process//DZDP//zhuozheng_cipinresult.csv')

# TF-IDF

In [91]:
text_str = str(text_join)

In [101]:
keywords = jieba.analyse.extract_tags(text_str, topK=500, withWeight=False, allowPOS=('n'))
print(keywords)

['景观', '风景区', '建筑', '用地', '城景', '风貌', '生态', '区域', '绿化', '山体', '居民点', '设施', '严格控制', '空间', '视廊', '环境', '特色', '服务设施', '资源', '片区', '保护区', '高度', '节点', '布局', '文化', '村庄', '体量', '总体规划', '环境保护', '关系', '植被', '道路', '功能', '措施', '视觉', '项目', '规模', '整体', '整治', '控制区', '色彩', '居民', '现状', '管理机构', '景点', '廊道', '绿地率', '界线', '污染环境', '产业', '重点', '功能定位', '民居', '协调控制', '地区', '利用', '红线', '建设项目', '水体', '目标', '分区', '世界遗产', '公路', '视线', '历史', '交通', '面积', '边界', '城区', '缓冲区', '陆域', '格局', '工业', '完整性', '强度', '单位', '环境质量', '国土', '村镇', '传统', '社区', '土地', '环境景观', '视域', '绿道', '海域', '绿地', '控制线', '背景', '建筑风格', '污染源', '构筑', '乡村', '城乡规划', '林地', '区域保护', '建筑物', '基础', '绿色', '地域', '度假区', '空间规划', '城市形象', '城市规划', '环境污染', '农业', '观光', '建议', '绿化带', '挖沙', '公共绿地', '基地', '内容', '植树造林', '公园', '类型', '人口', '规划区', '低层', '经济', '地质', '地貌', '污水', '流域', '林带', '农田', '容积率', '性质', '边缘', '密度', '水源', '号线', '森林', '保护措施', '一体', '斑块', '形式', '企业', '基础设施', '特征', '条件', '环节', '交融', '环境影响', '风格', '有序', '指导', '形态', '公益林', '轴线', '内应', '溪流', '分类', '营镇', '文物保护', '山地'

In [96]:
keywords = jieba.analyse.extract_tags(text_str, topK=500, withWeight=True, allowPOS=( 'n', 'v'))

In [102]:
keywords_df = pd.DataFrame(keywords)
keywords_df.to_csv(r'C:\Users\hai\Desktop\BaiduSyncdisk\1Review\Planning Manual\keywords_n.csv')

In [85]:
keywords2 = jieba.analyse.extract_tags(text_str, topK=10, withWeight=True, allowPOS=())
print(keywords2)

[('保护', 0.20942243214180892), ('外围', 0.16105978938161872), ('景观', 0.1603284685373432), ('地带', 0.14981174931652141), ('风景区', 0.140121583861145), ('建设', 0.11787267941557035), ('协调', 0.10980039800644), ('城市', 0.06973151522437371), ('控制', 0.06747367717910549), ('建筑', 0.061084303764305965)]


In [45]:
keywords3 = pd.DataFrame(keywords2)
# keywords3.to_csv('C://Users//hai//Desktop//text_process//DZDP//wangshi_ltresult.csv')

# PMI

In [46]:
word = {}   # 用来存储每个词在每一句中出现的次数
text_ls = list(set(text_ls))   # 去重
for k in text:
    for i in text_ls:
        if i in k:
            word[i] = word.get(i, 0) + 1
# word

In [47]:
# 计算共同出现的次数
name = '建设'
part_word = {}
num = 1
for i in text:
    for j in word:
        if (name in i) and (j in i):
            part_word[(name, j)] = part_word.get((name, j), 0) + 1
            num += 1
print(num)
print(type(part_word))
# part_word.keys()

16416
<class 'dict'>


In [48]:
alone_word = {}
for i in part_word.keys():
    a = word[i[0]] - part_word[i]    # 单词出现的总次数 - 共同出现的次数，即为这个单词单独出现的次数
    b = word[i[1]] - part_word[i]
    alone_word[i] = (a, b)
alone_word

{('建设', '保护地'): (8, 6),
 ('建设', '协调'): (7, 8),
 ('建设', '带'): (2, 10),
 ('建设', '用地'): (21, 4),
 ('建设', '并进'): (51, 0),
 ('建设', '禁止'): (27, 4),
 ('建设', '并'): (25, 3),
 ('建设', '应'): (11, 3),
 ('建设', '城市'): (20, 7),
 ('建设', '市'): (18, 8),
 ('建设', '区'): (0, 13),
 ('建设', '发展'): (20, 8),
 ('建设', '与'): (3, 11),
 ('建设', '划'): (3, 8),
 ('建设', '界'): (28, 5),
 ('建设', '风景区'): (18, 8),
 ('建设', '风景'): (4, 10),
 ('建设', '区外'): (22, 0),
 ('建设', '入'): (28, 1),
 ('建设', '外'): (1, 10),
 ('建设', '禁'): (15, 5),
 ('建设', '范'): (8, 3),
 ('建设', '围'): (0, 10),
 ('建设', '保'): (1, 13),
 ('建设', '地'): (1, 13),
 ('建设', '占'): (40, 4),
 ('建设', '建'): (0, 10),
 ('建设', '地带'): (5, 7),
 ('建设', '城'): (7, 8),
 ('建设', '严格'): (20, 2),
 ('建设', '风'): (4, 10),
 ('建设', '规'): (2, 8),
 ('建设', '外围'): (2, 9),
 ('建设', '侵占'): (43, 4),
 ('建设', '景区'): (7, 9),
 ('建设', '规划'): (5, 7),
 ('建设', '边界'): (44, 2),
 ('建设', '景'): (0, 12),
 ('建设', '保护'): (1, 12),
 ('建设', '建设'): (0, 0),
 ('建设', '用'): (14, 7),
 ('建设', '进'): (15, 8),
 ('建设', '域'): (13, 8),
 

In [49]:
pmi_dict = {}
for i in part_word.keys():
    a, b = alone_word[i]
    if (a * b) != 0:
        pmi_dict[i] = math.log2((part_word[i]* num) / (a * b)) 

In [50]:
l = list(pmi_dict.items())
l.sort(key= lambda x: x[1], reverse= True)
l[:300]

[(('建设', '外'), 16.381326638860784),
 (('建设', '保护'), 16.11829223302699),
 (('建设', '保'), 16.002815015607055),
 (('建设', '地'), 16.002815015607055),
 (('建设', '规'), 15.67524035757855),
 (('建设', '外围'), 15.505315356136236),
 (('建设', '带'), 15.353312262691187),
 (('建设', '求'), 15.34385193344212),
 (('建设', '要'), 15.10715167542179),
 (('建设', '分'), 15.085277175799026),
 (('建设', '划'), 15.061708704660623),
 (('建设', '于'), 14.962173031109709),
 (('建设', '范'), 14.909705611215573),
 (('建设', '绿'), 14.842350343413807),
 (('建设', '内'), 14.650513271676173),
 (('建设', '控'), 14.617524859722263),
 (('建设', '各'), 14.610497592828294),
 (('建设', '部'), 14.610497592828294),
 (('建设', '重'), 14.610497592828294),
 (('建设', '与'), 14.602277086023324),
 (('建设', '其'), 14.480862312411698),
 (('建设', '地带'), 14.458494499383244),
 (('建设', '规划'), 14.458494499383244),
 (('建设', '活动'), 14.386143655158559),
 (('建设', '破'), 14.386143655158559),
 (('建设', '公'), 14.386143655158559),
 (('建设', '应'), 14.35073831902736),
 (('建设', '风景'), 14.295596764